In [1]:
import pygmsh

In [2]:
resolution = 100

In [3]:
# Initialize empty geometry using the build in kernel in GMSH
geometry = pygmsh.geo.Geometry()
# Fetch model we would like to add data to
model = geometry.__enter__()

In [4]:
# Add points with finer resolution on left side
points = [model.add_point((0, 0, 0), mesh_size=resolution),
          model.add_point((5000, 0, 0), mesh_size=resolution),
          model.add_point((5000, 2000, 0), mesh_size=resolution),
          model.add_point((3000, 3000, 0), mesh_size=resolution),
#           model.add_point((5000, 3000, 0), mesh_size=resolution),
          model.add_point((0, 3000, 0), mesh_size=resolution)]

# Add lines between all points creating the rectangle
channel_lines = [model.add_line(points[i], points[i+1])
                 for i in range(-1, len(points)-1)]

# Create a line loop and plane surface for meshing
channel_loop = model.add_curve_loop(channel_lines)
plane_surface = model.add_plane_surface(channel_loop)

# Call gmsh kernel before add physical entities
model.synchronize()

In [5]:
geometry.generate_mesh(dim=2)
import gmsh
gmsh.write("p10_2.mesh")
gmsh.fltk.run()
gmsh.clear()
geometry.__exit__()

In [6]:
def create_mesh(mesh, cell_type, prune_z=False):
    cells = mesh.get_cells_type(cell_type)
#     cell_data = mesh.get_cell_data("gmsh:physical", cell_type)
    points = mesh.points[:,:2] if prune_z else mesh.points
    out_mesh = meshio.Mesh(points=points, cells={cell_type: cells})
    return out_mesh

In [7]:
import meshio
mesh_from_file = meshio.read("p10_2.mesh")

In [8]:
triangle_mesh = create_mesh(mesh_from_file, "triangle", prune_z=True)

In [9]:
meshio.write("p10_2.mesh", triangle_mesh)